#### Проблема независимых бинарных классификаторов
- Для каждой метки обучается отдельный бинарный классификатор
- Игнорируются зависимости между метками.

#### Classifier Chains (Цепочки классификаторов)
- Каждая метка прогнозируется с учётом предыдущих меток.
- Это позволяет учитывать зависимости между метками.

#### Label Powerset
- Каждая уникальная комбинация меток рассматривается как отдельный класс.
- Подходит, если меток немного и их комбинации ограничены.

#### Модели, поддерживающие мультилейбл классификацию напрямую
- Деревья решений (с параметрами мультилейбл в Scikit-learn).
- Алгоритмы, основанные на глубоких нейронных сетях.


In [4]:
from sklearn.datasets import make_multilabel_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, roc_auc_score
import numpy as np

X, Y = make_multilabel_classification(
    n_samples=1000, n_features=20, n_classes=5, n_labels=2, random_state=42)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

base_model = RandomForestClassifier(random_state=42)
chain_model = ClassifierChain(base_model, order='random', random_state=42)
chain_model.fit(X_train, Y_train)

probas = chain_model.predict_proba(X_test)
predictions = (probas > 0.5).astype(int)

hamming = hamming_loss(Y_test, predictions)
print(f"HammingLoss: {hamming:.4f}")

roc_auc = roc_auc_score(Y_test, probas, average='macro')
print(f"AUROC (macro): {roc_auc:.4f}")


HammingLoss: 0.1870
AUROC (macro): 0.8854


In [5]:
probas_single = chain_model.predict_proba([X_test[0]])[0]
top_10_indices = np.argsort(probas_single)[::-1][:10]
top_10_labels = top_10_indices
top_10_probabilities = probas_single[top_10_indices]

label_names = ['1', '2', '3', '4', '5']
top_10_named_labels = [label_names[idx] for idx in top_10_indices]

for label, prob in zip(top_10_named_labels, top_10_probabilities):
    print(f"Label: {label}, Probability: {prob:.4f}")

Label: 3, Probability: 0.9800
Label: 2, Probability: 0.2300
Label: 5, Probability: 0.1000
Label: 1, Probability: 0.0800
Label: 4, Probability: 0.0700
